In [ ]:
%%capture
!pip install pymorphy3 pymystem3 razdel spacy
!python -m spacy download ru_core_news_sm


In [ ]:
from pymorphy3 import MorphAnalyzer
from razdel import sentenize
from pymystem3 import Mystem
import spacy
import json

# **Задание 1**

In [ ]:
from razdel import sentenize

# sentenize не умеет в отсутствие пробелов для токенизации
text = 'I love my shoes...My shoes are beautiful.'
error_example = list(sentenize(text))
error_example


[Substring(0, 41, 'I love my shoes...My shoes are beautiful.')]

## **Задание 2**

In [ ]:
# задание 2

text = """
Вторым и третьим открытыми белыми карликами стали Сириус B и Процион B. В 1844 году директор Кёнигсбергской обсерватории Фридрих Бессель, анализируя данные наблюдений, которые велись с 1755 года, обнаружил, что Сириус, ярчайшая звезда земного неба, и Процион периодически, хотя и весьма слабо, отклоняются от прямолинейной траектории движения по небесной сфере[5]. Бессель пришёл к выводу, что у каждой из них должен быть близкий спутник. Сообщение было встречено скептически, поскольку слабый спутник оставался ненаблюдаемым, а его масса должна была быть достаточно велика — сравнимой с массой Сириуса и Проциона, соответственно.

В январе 1862 года Элвин Грэхэм Кларк, юстируя 18-дюймовый рефрактор, самый большой на то время телескоп в мире (Dearborn Telescope), впоследствии поставленный семейной фирмой Кларков в обсерваторию Чикагского университета, обнаружил в непосредственной близости от Сириуса тусклую звёздочку. Это был спутник Сириуса, Сириус B, предсказанный Бесселем[6]. А в 1896 году американский астроном Д. М. Шеберле открыл Процион B, подтвердив тем самым и второе предсказание Бесселя.

В 1915 году американский астроном Уолтер Сидней Адамс измерил спектр Сириуса B. Из измерений следовало, что его температура не ниже, чем у Сириуса A (по современным данным, температура поверхности Сириуса B составляет 25 000 K, а Сириуса A — 10 000 К), что, с учётом его в 10 000 раз меньшей, чем у Сириуса A, светимости указывает на очень малый радиус и, соответственно, высокую плотность — 106 г/см3 (плотность Сириуса ~0,25 г/см3, плотность Солнца ~1,4 г/см3).
"""

from razdel import tokenize as razdel_tokenize
razdel_raw = list(razdel_tokenize(text))
razdel_results = []
for i in razdel_raw:
  razdel_results.append(i.text)

mystem = Mystem(disambiguation=False)
mystem_raw = mystem.analyze(text)
mystem_results = []
for i in mystem_raw:
  mystem_results.append(i['text'])

# как можно увидеть из работы mystem, он токенизирует в т. ч. пробелы
# и отступы строк, что вообще-то не очень хорошо; давайте уберем и сравним
# результат

mystem_results_without_spaces = []
for i in mystem_results:
  if i != '\n' and i != ' ':
    mystem_results_without_spaces.append(i.strip())

# сравниваем результат; также отметим, что длина списка
# mystem_results_without_spaces = 293, в то время как у razdel_results = 292;
# это значит, что mystem токенизировал как минимум на один символ больше
for i in range(len(razdel_results)):
  if mystem_results_without_spaces[i] != razdel_results[i] and i < 291:
    print(i, mystem_results_without_spaces[i], razdel_results[i], sep='|')

116|18|18-дюймовый
117|-|рефрактор
118|дюймовый|,
119|рефрактор|самый
120|,|большой
121|самый|на
122|большой|то
123|на|время
124|то|телескоп
125|время|в
126|телескоп|мире
127|в|(
128|мире|Dearborn
129|(|Telescope
130|Dearborn|)
131|Telescope|,
132|),|впоследствии
133|впоследствии|поставленный
134|поставленный|семейной
135|семейной|фирмой
136|фирмой|Кларков
137|Кларков|в
138|в|обсерваторию
139|обсерваторию|Чикагского
140|Чикагского|университета
141|университета|,
142|,|обнаружил
143|обнаружил|в
144|в|непосредственной
145|непосредственной|близости
146|близости|от
147|от|Сириуса
148|Сириуса|тусклую
149|тусклую|звёздочку
150|звёздочку|.
151|.|Это
152|Это|был
153|был|спутник
154|спутник|Сириуса
155|Сириуса|,
156|,|Сириус
157|Сириус|B
158|B|,
159|,|предсказанный
160|предсказанный|Бесселем
161|Бесселем|[
162|[|6
163|6|]
164|]|.
165|.|А
166|А|в
167|в|1896
168|1896|году
169|году|американский
170|американский|астроном
171|астроном|Д
172|Д|.
173|.|М
174|М|.
175|.|Шеберле
176|Шеберле|открыл
177|от

Как можно увидеть, проблема возникла с токенизацией "18-дюймовый": MyStem посчитал это слово как три токена, в то время как Razdel интерпретировал его как один; при этом в 132 элементе MyStem ***), ***слепил в один токен, а Razdel их разделяет (извините за каламбур). Все это подводит нас к выводу, что razdel все же использовать более продуктивно, чем mystem:
- меньше количество учитываемых факторов при извчелении токенов (что можно увидеть даже по количеству строк, которое я посвятила каждому из них)
- более понятные "правила игры": не очень понятно, в каких случаях MyStem интерпретирует знаки препинания как отдельные токены, а когда не разделяет две разные сущности.

# **Задание 3**

In [ ]:
mystem_lem = mystem.lemmatize(text)

morph = MorphAnalyzer()
pymorphy_lem_raw = [morph.parse(i) for i in mystem_results]
pymorphy_lem = []
for i in range(len(pymorphy_lem_raw)):
  pymorphy_lem.append(pymorphy_lem_raw[i][0].normal_form)

# сравниваем
mst_pym = 0
for i in range(len(pymorphy_lem)):
  if mystem_lem[i] != pymorphy_lem[i]:
    print(i, mystem_lem[i], pymorphy_lem[i], sep='|')
  if mystem_lem[i] == pymorphy_lem[i]:
    mst_pym += 1
print('Общее у pymorphy и mystem: ', round(mst_pym/len(pymorphy_lem)*100, 2),
      '%')

13|становиться|стать
17|B|b
23|B|b
33|кенигсбергский|кёнигсбергский
43|данные|дать
57|обнаруживать|обнаружить
106|приходить|прийти
134|встречать|встретить
190|грэхэм|грэхэма
218|Dearborn|dearborn
220|Telescope|telescope
224|поставлять|поставить
240|обнаруживать|обнаружить
254|звездочка|звёздочка
266|B|b
268|предсказывать|предсказать
287|д|далее
291|шеберля|шеберл
293|открывать|открыть
297|B|b
301|то|тем
307|второй|второе
331|измерять|измерить
337|B|b
353|низко|ниже
361|A|a
375|B|b
383|K|k
389|A|a
401|учет|учёт
413|меньший|малый
421|A|a
Общее у pymorphy и mystem:  93.26 %


**Проблемы MyStem:**

*  склонен интерпретировать глаголы совершенного вида как глаголы несовершенного вида (13: *стали* как *становиться*; 240: *обнаружил* - как *обнаруживать* и т. д.)
*   исчезновение буквы ё (401: *учет*), что в принципе не такая уж проблема, но может ею стать, если мы будем сталкиваться с неоднозначными кейсами

**Проблемы PyMorphy:**
*   иногда мудрит и делает слишком много работы (типа приведения к нижнему регистру букв у звезд или лемматизация *меньший* как *малый*, хотя это достаточно спорный компаратив; с именами та же проблема)
*   иногда, наоборот, не доделывает необходимый минимум: явный компаратив *ниже* у него остался без изменений

Оба товарища не справились с фамилиями Грэхема и Шеребле. Возможно, есть еще формы, которые они интерпретировали неверно, но одинаково.


# **Задание 4**

In [ ]:
def mystem_tokenizer(text):
    tokens = []
    for token in mystem.analyze(text):
      tokens.append(token["text"])
    return spacy.tokens.Doc(nlp.vocab, tokens)

nlp.tokenizer = mystem_tokenizer
spacy_res = nlp(text)
spacy_lem = []
for sent in spacy_res.sents:
    for token in sent:
        spacy_lem.append(token.lemma_.lower())

# сравнение
sp_pym = 0
sp_mst = 0
for i in range(len(spacy_lem)):
  if mystem_lem[i] != spacy_lem[i] or spacy_lem[i]!=pymorphy_lem[i]:
    print(i, mystem_lem[i], pymorphy_lem[i], spacy_lem[i], sep='|')
  if mystem_lem[i] == spacy_lem[i]:
    sp_mst += 1
  if pymorphy_lem[i] == spacy_lem[i]:
    sp_pym += 1

print('Общее у SpaCy и MyStem: ', round(sp_mst/len(spacy_lem)*100, 2), '%',
      '\n',
      'Общее у SpaCy и PyMorphy: ', round(sp_pym/len(spacy_lem)*100, 2), '%',
      '\n',
      'Общее у PyMorphy и MyStem: ', round(mst_pym/len(pymorphy_lem)*100, 2),
      '%',
      sep='')

13|становиться|стать|стать
17|B|b|b
23|B|b|b
33|кенигсбергский|кёнигсбергский|кёнигсбергский
43|данные|дать|данные
47|который|который|которые
57|обнаруживать|обнаружить|обнаружить
63|яркий|яркий|ярчайшая
106|приходить|прийти|прийти
120|они|они|них
132|быть|быть|было
134|встречать|встретить|встретить
174|процион|процион|проциона
190|грэхэм|грэхэма|грэхэм
208|то|то|тот
218|Dearborn|dearborn|dearborn
220|Telescope|telescope|telescope
224|поставлять|поставить|поставить
230|кларк|кларк|кларков
236|чикагский|чикагский|чикагского
240|обнаруживать|обнаружить|обнаружить
254|звездочка|звёздочка|звёздочка
266|B|b|b
268|предсказывать|предсказать|предсказанный
270|бессель|бессель|бесселем
287|д|далее|д
291|шеберля|шеберл|шеберле
293|открывать|открыть|открыть
297|B|b|b
301|то|тем|тем
303|самый|самый|самым
307|второй|второе|второй
311|бессель|бессель|бесселя
331|измерять|измерить|измерить
337|B|b|b
353|низко|ниже|низкий
361|A|a|a
367|данные|данные|данным
375|B|b|b
383|K|k|k
389|A|a|a
401|учет|учёт|уч

Выводы по сравнению:


*   SpaCy больше похож на PyMorphy: в плюсах та же буква ё и умение в вид глагола
* При этом SpaCy поправил косяк с *низким-ниже*
*   SpaCy, судя по всему, не очень могет в падежные окончания, потому что некоторые штуки он вообще оставил без лемматизации (типа *чикагского*)
*  SpaCy справился с Шеребле!